## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [103]:
from gensim.models import Word2Vec, KeyedVectors
from gensim import matutils
import re
import os
from tqdm import tqdm_notebook as tqdm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument
import nltk
import pickle

import numpy as np
import json

import operator

In [ ]:
with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [6]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [191]:
qa_corpus[0]

['добрый день мой сын гражданин украины днр имеет вид на жительство в р ф кот получил проживая с г в нижегородской области в г переехал на постоянное место жительство в г ростов официально трудоустроился на одно из промышл предприятий г ростова оформил временную регистрацию в ростове в уфмс предупредили что по истечении дней он должен либо постоянно прописаться либо покинуть территорию россии прошу проконсультировать как быть дальше вернуться домой в донецк но здесь идет война работы нет в ростове он работает по специальности он инженер машиностроитель временная прописка до марта если он сможет приобрести какую либо недвижимость как долго будет решаться вопрос о его постоянной прописке в ростове как в этом случае будет решаться вопрос с видом на жительство в ростове не получится ли что приобретя квартиру он не успеет в ней прописаться до окончании срока временной регистрации с уважением людмила евгеньевна',
 'добрый вечер из вашего вопроса вообще ничего не ясно ваш сын по внж в нижегор

In [252]:
def preprocessing(par):
    par = par.replace('.', ' ')
    par = par.replace(',', ' ')
    par = par.replace(':', ' ')
    par = par.replace(';', ' ')
    par = par.replace('-', ' ')
    par = par.replace(')', ' ')
    par = par.replace('(', ' ')
    par = par.replace('\\', ' ')
    par = par.replace('/', ' ')
    par = par.replace('"', ' ')
    par = par.replace('!', ' ')
    par = par.replace('?', ' ')
    par = par.replace('\n', ' ')
    par = par.replace('\xa0', ' ')
    par = par.replace('\u200b', ' ')
    par = par.replace('\r', ' ')
    par = re.sub('[0-9]','', par)
    par = par.replace('   ', ' ')
    par = par.replace('  ', ' ')
    par = par.strip(' ')
    par = par.lower()
        
    return par

In [199]:
#dict_qa = {}
questions = []
answers = []
for i in range(0, len(qa_corpus)):
    questions.append(preprocessing(qa_corpus[i][0]))
    answers.append(preprocessing(qa_corpus[i][1]))
    #dict_qa[qa_corpus[i][0]] = qa_corpus[i][1]

In [196]:
qa_corpus[0]

['добрый день мой сын гражданин украины днр имеет вид на жительство в р ф кот получил проживая с г в нижегородской области в г переехал на постоянное место жительство в г ростов официально трудоустроился на одно из промышл предприятий г ростова оформил временную регистрацию в ростове в уфмс предупредили что по истечении дней он должен либо постоянно прописаться либо покинуть территорию россии прошу проконсультировать как быть дальше вернуться домой в донецк но здесь идет война работы нет в ростове он работает по специальности он инженер машиностроитель временная прописка до марта если он сможет приобрести какую либо недвижимость как долго будет решаться вопрос о его постоянной прописке в ростове как в этом случае будет решаться вопрос с видом на жительство в ростове не получится ли что приобретя квартиру он не успеет в ней прописаться до окончании срока временной регистрации с уважением людмила евгеньевна',
 'добрый вечер из вашего вопроса вообще ничего не ясно ваш сын по внж в нижегор

In [138]:
def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    sim = np.dot(v1_norm, v2_norm)
    if sim is not None:
        return sim
    else:
        return 0

### W2V

In [22]:
model_path = r"C:\Users\1\Downloads\all.norm-sz100-w10-cb0-it1-min100.w2v"
model_w = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

In [58]:
def par_t_v(par):
    vecs = np.zeros(100)
    par = list(filter(None, par.split(' ')))
    for word in par:
        try:
            vec = model_w.get_vector(word)
        except Exception as ex:
            continue
        if vecs.size is 0:
            vecs = vec
        else:
            vecs = vecs+vec
    vec = vecs/len(par)
    return vec

In [299]:
def get_w2v_vectors(pars):
    pars_and_vecs = {}
    for i in range(0, len(pars)):
        par = preprocessing(pars[i])
        vec_par = par_t_v(par)
        pars_and_vecs[i] = [vec_par, pars[i]]
    return pars_and_vecs

In [302]:
q_vecs_w = get_w2v_vectors(questions)

In [303]:
a_vecs_w = get_w2v_vectors(answers)

In [318]:
def search_w2v(query):
    global q_vecs_w
    query = preprocessing(query).split(' ')
    vec = np.zeros(100)
    for word in query:
        try:
            v = model_w.get_vector(word)
            vec += v
        except Exception as ex:
            continue
    doc2similar = {}
    for i in range(0, len(q_vecs_w)):
            doc2similar[i] = similarity (vec, q_vecs_w[i][0])
    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    top = sorted_dict[:5]
    return top

### D2V

In [63]:
model_d = Doc2Vec.load('Doc2Vec_100s_1000e')

In [66]:
def doc_t_v(par):
    vec = np.zeros(100)
    try:
        vec = model_d.infer_vector(par)
    except Exception as ex:
        print(ex)
    return vec

In [148]:
def get_d2v_vectors(pars):
    pars_and_vecs = {}
    for i in range(0, len(pars)):
        par = preprocessing(pars[i])
        vec_par = doc_t_v(par)
        pars_and_vecs[i] = [vec_par, pars[i]]
    return pars_and_vecs

In [203]:
q_vecs_d = get_d2v_vectors(questions)

In [204]:
a_vecs_d = get_d2v_vectors(answers)

In [253]:
def search_d2v(query):
    global q_vecs_d
    query = preprocessing(query)
    vec = np.zeros(100)
    vec = model_d.infer_vector(query)
    doc2similar = {}
    for i in range(0, len(q_vecs_d)):
            doc2similar[i] = similarity (vec, q_vecs_d[i][0])
    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    top = sorted_dict[:5]
    return top
    

In [286]:
a=['saf', 'aewf']
for idx, b in enumerate(a):
    idx == top[4][0]

### Checking

In [262]:
def search(query, search_method):
    if search_method == 'word2vec':
        search_result = search_w2v(query)
    elif search_method == 'doc2vec':
        search_result = search_d2v(query)
    else:
        raise TypeError('unsupported search method')
    return search_result

In [296]:
def check(method):
    global questions
    true = 0
    for idx, question in tqdm(enumerate(questions)):
        top = search(question, method)
        for i in range(0, 5):
            if idx == top[i][0]:
                true += 1
    check = true / len(questions)
    return check

In [297]:
a = check('doc2vec')

A Jupyter Widget

In [298]:
a

0.8157514450867052

In [319]:
b = check('word2vec')

A Jupyter Widget

In [320]:
b

1.0